In [3]:
import requests
from bs4 import BeautifulSoup

URL = "https://companiesmarketcap.com/usa/largest-companies-in-the-usa-by-market-cap/"

# Scrape page
r = requests.get(URL, headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(r.text, "html.parser")

symbols, names, prices = [], [], []

rows = soup.select("table tbody tr")
for row in rows:
    name_tag = row.select_one("div.company-name")
    symbol_tag = row.select_one("div.company-code")
    cols = row.find_all("td")

    if not name_tag or not symbol_tag or len(cols) < 5:
        continue

    name = name_tag.get_text(strip=True)
    symbol = symbol_tag.get_text(strip=True)
    price_str = cols[4].get_text(strip=True).replace("$", "").replace(",", "")

    try:
        price = float(price_str)
    except:
        price = None

    names.append(name)
    symbols.append(symbol)
    prices.append(price)

    if len(names) == 100:  # only top 100
        break

print("Symbols:", symbols[:5], "...")
print("Names:", names[:5], "...")
print("Prices:", prices[:5], "...")


Symbols: ['NVDA', 'MSFT', 'AAPL', 'GOOG', 'AMZN'] ...
Names: ['NVIDIA', 'Microsoft', 'Apple', 'Alphabet (Google)', 'Amazon'] ...
Prices: [176.67, 517.93, 245.5, 255.24, 231.48] ...


In [18]:
import yfinance as yf
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

results = []

for idx,ticker in enumerate(symbols):
    try:
        data = yf.download(ticker, period="20d", interval="1d", progress=False)
        # data = data.iloc[:-1]

        if data.empty:
            continue

        # Round values
        data = data.round(2)

        # 20-day high/low
        high_20 = float(data["High"].max())
        low_20 = float(data["Low"].min())

        # Latest close = current price
        current_price = prices[idx]

        results.append([ticker, current_price, high_20, low_20])

    except Exception as e:
        print(f"⚠️ Skipping {ticker}: {e}")

# Create DataFrame
df = pd.DataFrame(results, columns=["Ticker", "Current Price", "20D High", "20D Low"])


In [19]:
df


,Ticker,Current Price,20D High,20D Low
0,NVDA,176.67,184.46,164.06
1,MSFT,517.93,517.23,492.37
2,AAPL,245.50,241.32,224.69
3,GOOG,255.24,254.14,201.95
4,AMZN,231.48,238.85,220.82
...,...,...,...,...
95,WELL,165.19,170.99,163.97
96,LMT,472.94,478.44,441.64
97,MO,64.89,67.51,63.94
98,PLD,113.51,116.41,108.18


In [20]:
# Stocks where Current Price > 20D High
above_high = df[df["Current Price"] > df["20D High"]]["Ticker"].tolist()

# Stocks where Current Price < 20D Low
below_low = df[df["Current Price"] < df["20D Low"]]["Ticker"].tolist()

print("Above 20D High:", above_high)
print("Below 20D Low:", below_low)


Above 20D High: ['MSFT', 'AAPL', 'GOOG', 'JPM', 'PLTR', 'ABBV', 'BAC', 'GE', 'WFC', 'IBM', 'ABT', 'APP', 'APH', 'PANW', 'HOOD']
Below 20D Low: ['DIS', 'HON', 'NKE']
